In [1]:
import twitter

In [2]:
consumer_key = 'WlwoXrPuQKQHWtUeLrB1BnYhG'
consumer_secret = 'ZgUyr2mxF0W7xDl9fuqojCjFjqJYIuoKj0e8o8HvtFpy5tqDIq'
access_token = '3860491152-JgSlqASKgZqzxQp8pZs3g5ewWOJVC9ts4qgeciY'
access_secret = 'itK8tyL1JUmSw8doHILhLNHbFmH0D3MJIe8bhf1TT8guN'

In [3]:
api = twitter.Api(consumer_key, consumer_secret, access_token, access_secret, sleep_on_rate_limit=True)

In [6]:
import json

with open("User_obj.txt", mode='w') as file:
    usr = api.GetUser(user_id=100000021)
    usr_str = str(usr._json)
    file.write(usr_str)

In [ ]:
user_list = api.UsersLookup(user_id=[943015703723905024,])
user_list[0]._json

In [2]:
import sqlite3 as sql
conn = sql.connect('twitter_handles.db')
c = conn.cursor()

In [3]:
c.execute("ALTER TABLE handles RENAME TO temp_handles")

In [30]:
#c.execute("CREATE TABLE temp (ID INTEGER PRIMARY KEY, User_object TEXT)")
c.execute('''CREATE TABLE handles_join
             (ID INTEGER PRIMARY KEY, 
              Follower_Source TEXT, 
              Follower_IDs TEXT UNIQUE,
              User_Object TEXT);''')

In [5]:
c.execute('''INSERT INTO handles
             SELECT ID, Follower_Source, Follower_IDs
             FROM temp_handles''')

In [33]:
c.execute("DROP TABLE temp_handles")

In [34]:
conn.commit()

In [28]:
c.execute('''SELECT temp_handles.Follower_Source, temp_handles.Follower_IDs, temp.User_Object
                        FROM temp_handles
                        LEFT JOIN temp ON temp_handles.ID = temp.ID''')
viewData = c.fetchall()

c.execute("INSERT INTO handles_join(Follower_Source, Follower_IDs, User_Object) VALUES(?,?,?)", (row[0], row[1], row[2],))

[('_MediBloc', '829892082588282880', '{"id": 829892082588282880, "id_str": "829892082588282880", "name": "\\ucd5c\\uc6d0\\uc11d", "screen_name": "red8ull87", "location": "", "description": "", "url": null, "entities": {"description": {"urls": []}}, "protected": false, "followers_count": 0, "friends_count": 29, "listed_count": 0, "created_at": "Fri Feb 10 03:17:45 +0000 2017", "favourites_count": 4, "utc_offset": null, "time_zone": null, "geo_enabled": false, "verified": false, "statuses_count": 1, "lang": "ko", "contributors_enabled": false, "is_translator": false, "is_translation_enabled": false, "profile_background_color": "F5F8FA", "profile_background_image_url": null, "profile_background_image_url_https": null, "profile_background_tile": false, "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2"

In [31]:
for data in viewData:
    c.execute("INSERT INTO handles_join(Follower_Source, Follower_IDs, User_Object) VALUES(?,?,?)", (data[0], data[1], data[2],))
    

In [17]:
for row in c.execute("SELECT * FROM temp_handles WHERE ID < 10"):
    print(row)

(1, '_MediBloc', '829892082588282880', None)
(2, '_MediBloc', '943690737190703105', None)
(3, '_MediBloc', '943686183745228801', None)
(4, '_MediBloc', '942799098050068480', None)
(5, '_MediBloc', '935683612132773888', None)
(6, '_MediBloc', '394962803', None)
(7, '_MediBloc', '943492366119288832', None)
(8, '_MediBloc', '943659202286727168', None)
(9, '_MediBloc', '2898745885', None)


In [32]:
for row in c.execute("SELECT * FROM handles_join WHERE ID < 10"):
    print(row)

(1, '_MediBloc', '829892082588282880', '{"id": 829892082588282880, "id_str": "829892082588282880", "name": "\\ucd5c\\uc6d0\\uc11d", "screen_name": "red8ull87", "location": "", "description": "", "url": null, "entities": {"description": {"urls": []}}, "protected": false, "followers_count": 0, "friends_count": 29, "listed_count": 0, "created_at": "Fri Feb 10 03:17:45 +0000 2017", "favourites_count": 4, "utc_offset": null, "time_zone": null, "geo_enabled": false, "verified": false, "statuses_count": 1, "lang": "ko", "contributors_enabled": false, "is_translator": false, "is_translation_enabled": false, "profile_background_color": "F5F8FA", "profile_background_image_url": null, "profile_background_image_url_https": null, "profile_background_tile": false, "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F

In [6]:
import json
a = 0
while True:
    row_list = []
    for row in c.execute("SELECT * FROM handles LIMIT " + str(a) + ", 100"):
        row_list.append(row[2])
    
    user_100 = api.UsersLookup(row_list)
    for user in user_100:
        
        user = user._json
        encode_user = json.JSONEncoder().encode(user)
        c.execute("INSERT INTO handles(User_Object) VALUES (?)", (encode_user,))

    a += 100
    conn.commit()
    print(str(a) + "\t")
        

IntegrityError: NOT NULL constraint failed: handles.Follower_Source

In [23]:
conn.commit()

In [13]:
import csv
with open("user_9.csv", mode='w', newline='') as csvfile:
    write = csv.writer(csvfile)
    for row in c.execute("SELECT * FROM handles_join WHERE ID >= 800000"):
        write.writerow(row)